In [0]:
DEBUG = False
CONFIG_FILE = '/datascope/subaru/data/targeting/dSph/ursaminor/netflow/SSP/ursaminor_6_013/ga-netflow_20250310115104.config'
OUTPUT_PATH = '/datascope/subaru/data/targeting/dSph/ursaminor/netflow/SSP/ursaminor_6_013'

In [0]:
import os, sys
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Ellipse, Circle
from matplotlib.gridspec import GridSpec
import commentjson as json

In [0]:
plt.rc('font', size=6)

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
if DEBUG and 'debug' not in globals():
    import debugpy
    debugpy.listen(('0.0.0.0', 5698))
    debug = True

# Imports

In [0]:
import pfs.utils
from pfs.datamodel import TargetType

from pfs.ga.targeting.config.netflow import NetflowConfig
from pfs.ga.targeting.scripts.netflow.netflowscript import NetflowScript
from pfs.ga.targeting.io import DataFrameSerializer, ObservationSerializer
from pfs.ga.targeting.targets.dsph import *
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, FOV, FP, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color
from pfs.ga.targeting.projection import WcsProjection, Pointing
from pfs.ga.targeting.netflow import Netflow
from pfs.ga.targeting.io import DataFrameSerializer
from pfs.ga.targeting.targets.dsph import GALAXIES as DSPH_FIELDS
from pfs.ga.targeting.targets.m31 import M31_FIELDS

# Load the netflow config file and args file

In [0]:
# Load the configuration
config = NetflowConfig.default()
config.load(CONFIG_FILE, ignore_collisions=True, format='.json')

print(config.targets.keys())

In [0]:
config.pointings

In [0]:
with open(os.path.splitext(CONFIG_FILE)[0] + '.args', 'r') as f:
    args = json.load(f)

In [0]:
args

# Load the target lists

In [0]:
target_lists = {}

for key in config.targets:
    fn = os.path.join(OUTPUT_PATH, f'{config.field.key}_targets_{key}.feather')
    target_lists[key] = NetflowScript.load_target_list(key, config.targets[key], fn)
    print(key, config.targets[key].prefix, config.targets[key].path)

# Load the assignments

In [0]:
# Load the assignments

fn = os.path.join(OUTPUT_PATH, f'{config.field.key}_assignments_all.feather')
assignments = DataFrameSerializer().read(fn)

print(assignments.shape)
for c in assignments.columns:
    print(c, assignments[c].dtype)

In [0]:
if 'dsph' in args:
    field = DSPH_FIELDS[args['dsph']]
else:
    raise NotImplementedError()

field

In [0]:
hsc = field.get_photometry()
cmd = field.get_cmd()
ccd = field.get_ccd()

In [0]:
pointing = field.get_center()
pointing

In [0]:
wcs = WcsProjection(pointing, proj='TAN')
fov = FOV(projection=wcs)

# Calculate the whitening matrix

This basically determines the second moments of the coordinate distribution
and yield the axes of the ellipsoid and the position angle

In [0]:
from pfs.ga.targeting.projection import Whitening
from pfs.ga.targeting.radialprofile import King

In [0]:
# obs = target_lists[list(target_lists.keys())[0]]
obs = target_lists['hsc']

In [0]:
# Iteratively determine a whitening matrix that converts the
# spherical coordinates into deprojected coordinates

w = Whitening(projection=wcs)
w_mask = w.create(obs, iterations=100, s_cut=2.3)

w.M, w.reverse(w.M), field.pos

In [0]:
[dsph_maj, dsph_min] = w.S
dsph_pa = np.degrees(np.angle(w.Vh[0, 0] + 1j * w.Vh[1, 0]))
dsph_maj, dsph_min, dsph_pa

In [0]:
f, ax = plt.subplots(1, 1, figsize=(8, 8), dpi=240)

(wx, wy) = w.apply(obs.get_coords())
ax.plot(wx, wy, '.', ms=0.1)
ax.plot(wx[w_mask], wy[w_mask], '.', ms=0.1)
ax.set_aspect('equal')
ax.grid()

In [0]:
w.M, w.W, w.reverse(w.M)

In [0]:
# Fit profile to all stars within the color cut

king = King(transformation=w, R_max=7.5, bins=30)
king.histogram(obs) #, mask=w_mask)
[S_b, S_0, R_c, R_t], pcov = king.fit()

S_b, S_0, R_c, R_t

In [0]:
# Verify the King profile fit

f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=120)

R, log_S, log_S_sigma = king.get_log_S()

ax.errorbar(R, log_S, 3 * log_S_sigma)
ax.set_xlabel('deprojected R')
ax.set_ylabel(r'$\log \Sigma$')

R = np.linspace(0, 7.5, 200)
log_S = king.log_eval(R)
ax.plot(R, log_S)

# Define pointings

In [0]:
from astropy.time import Time

In [0]:
center = field.get_center()
obs_time = Time('2025-03-25T10:00:00')        # midnight in Hawaii

# Use the field defaults
# pointings = field.get_pointings(SubaruPFI)

# Use the config file
pointings = [ p.get_pointing(obs_time=obs_time) for p in config.pointings ]


# Plot

In [0]:
f, ax = plt.subplots(1, 1, figsize=(8, 8), dpi=240, subplot_kw=dict(projection=wcs.wcs))

fov.plot_observation(ax, obs, c='black', size=0.5)

# Plot the tidal radius from the King profile fit
ell = king.get_ellipse(R_t)
fov.plot(ax, ell, native_frame='world', ls='--', lw=1, c='k')
ell = king.get_ellipse(R_t * 2)
fov.plot(ax, ell, native_frame='world', ls='--', lw=0.5, c='k')
ell = king.get_ellipse(R_t * 3)
fov.plot(ax, ell, native_frame='world', ls='--', lw=0.5, c='k')
ell = king.get_ellipse(R_c)
fov.plot(ax, ell, native_frame='world', ls='-', lw=1, c='k')

# Plot the poiting center
# fov.plot(ax, [field.pos.ra.value], [field.pos.dec.value], fmt='+r', ms=10, native_frame='world')

for pp in pointings:
    wfc = SubaruWFC(pp)
    pfi = SubaruPFI(wfc, instrument_options=config.instrument_options)

    pfi.plot_focal_plane(ax, fov, corners=True, c='r', ls='-', lw=1)
    pfi.plot_focal_plane(ax, fov, fill=True, alpha=0.2)

ax.set_xlim(1, -3)
ax.set_ylim(-3, 1)
ax.grid()

# Plot the radial distribution of the assigned targets

In [0]:
R_c, R_t

In [0]:
f, ax = plt.subplots(1, 1, figsize=(4, 3), dpi=240)

# Non-filler targets
mask = (assignments['prefix'] == 'sci') & ((assignments['priority'] < 5) | (assignments['priority'] == 8))
mask &= assignments[mask].duplicated(['__target_idx'], keep='first')
(x, y) = w.apply(assignments['RA'][mask], assignments['Dec'][mask])
r = np.sqrt(x**2 + y**2)
hist, bins = np.histogram(r / R_t, bins=100)
ax.step(0.5 * (bins[1:] + bins[:-1]), hist, where='mid', label='non filler')

# All science targets
mask = (assignments['prefix'] == 'sci')
mask &= assignments[mask].duplicated(['__target_idx'], keep='first')
(x, y) = w.apply(assignments['RA'][mask], assignments['Dec'][mask])
r = np.sqrt(x**2 + y**2)
hist, bins = np.histogram(r / R_t, bins=100)
ax.step(0.5 * (bins[1:] + bins[:-1]), hist, where='mid', lw=0.5, label='all science')

ax.set_xlabel('$R / R_T$')
ax.set_ylabel('assigned target count')

ax.set_xlim(0, 3.25)
ax.set_ylim(0, 1800)

ax.set_title('Distribution of assigned targets according to elliptical distance')
ax.legend()